In [ ]:
import sympy as sp
from sympy import symbols
from sympy.matrices import Matrix

sp.init_printing(use_unicode=True)

In [ ]:
# The number of arbitrageurs is set as exogenous variable N

We start with the case such that the number of arbitrageurs against the pool is exogenously set by $ N $. Let $ P $ be the price of external markets, such as CEXs, $ \gamma $ be the fee rate, $ X $ and $ Y $ be the reserves of pool. The spot price of pool is then $ \frac{Y}{X} $. Since we will take $ \gamma $ of both input and output token, effective fee rate will be around double of it. Letting $ (x_1, y_1), (x_2, y_2), \cdots , (x_N, y_N) $ be pair of each player's input amounts, the clearing price $ P_c $ will be following: 
$$
\begin{align} 
P_c = \frac{Y + 2\sum^N_{i=1} y_i }{X + 2\sum^N_{i=1} x_i} 
\end{align}
$$

Each player $ i $'s revenue $ R_i $ will be:
$$
R_i = -(1 + \gamma)(P x_i + y_i) + (1 - \gamma)(\frac{P}{P_c}y_i + P_c x_i)
$$

It is straight forward to see that the best response of $ i $ should be either $ (x_i, 0) $ or $ (0,y_i) $. WLOG let's assume $ \frac{1 - \gamma}{1 + \gamma}P > \frac{Y}{X} $ and each player's best response can be identified by $ y_i $.

From the first order condition, while $ P_{-i} $ is defined as clearing price without including bid from player $ i $, we have following equation:
$$
P_c = \sqrt{\frac{1 - \gamma}{1 + \gamma}P P_{-i}}
$$

Note that this holds for every $ i,j \in [N] $ thus
$$
P_c = \sqrt{\frac{1 - \gamma}{1 + \gamma}P P_{-i}} = \sqrt{\frac{1 - \gamma}{1 + \gamma}P P_{-j}},
$$
and the pure strategy Nash equilibrium is unique and symmetric.

Let $ y_\text{eq} $ be the equilibrium bid of each player. 